In [1]:
pip install datasets


In [2]:
!pip install --upgrade datasets
!pip install --upgrade fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [3]:
from datasets import load_dataset


# Load with streaming (does not cache to local file system)
ds = load_dataset("ServiceNow/repliqa", split="repliqa_4", streaming=True)

# Gather one-word QAs
qa_list = []
for example in ds:
    ans = example["answer"]
    if (
        isinstance(ans, str)
        and ans.strip() != ""
        and " " not in ans.strip()
    ):
        qa_list.append({"question": example["question"], "answer": ans.strip()})
        if len(qa_list) == 100:
            break

print(qa_list[:3])


# Print a sample of 5 QA pairs
for i in range(5):
    print(f"Q: {qa_list[i]['question']}")
    print(f"A: {qa_list[i]['answer']}\n")


[{'question': 'In what month does the Old Town Parade and Heritage Festival take place?', 'answer': 'December.'}, {'question': 'What environmental threat is the community of Tamarist Village organizing against, as outlined in the document?', 'answer': 'Desertification.'}, {'question': 'In what year did robotics and coding clubs become part of the curriculum at Summit Valley School?', 'answer': '2030'}]
Q: In what month does the Old Town Parade and Heritage Festival take place?
A: December.

Q: What environmental threat is the community of Tamarist Village organizing against, as outlined in the document?
A: Desertification.

Q: In what year did robotics and coding clubs become part of the curriculum at Summit Valley School?
A: 2030

Q: What is the minimum fundraising commitment for teams participating in the 'Great Fall Relay'?
A: $500.

Q: What percentage of SMEs have faced significant supply chain disruptions since mid-2023 according to Dr. Emily Sanders' survey?
A: 73%



In [ ]:
import json

qa_list = [{"question": ex["question"], "answer": ex["answer"]} for ex in qa_list]
with open("repliqa_one_word_qa.json", "w", encoding="utf-8") as f:
    json.dump(qa_list, f, ensure_ascii=False, indent=2)


In [3]:
from datasets import load_dataset

# Load RepLiQA
ds = load_dataset("ServiceNow/repliqa", split="repliqa_4")

# Filter to one-word answers
def is_one_word(example):
    return (
        isinstance(example["answer"], str) and
        example["answer"].strip() != "" and
        " " not in example["answer"].strip()
    )

one_word_ds = ds.filter(is_one_word)
print(f"One-word Q&A pairs found: {len(one_word_ds)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/repliqa_0-00000-of-00001.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

data/repliqa_1-00000-of-00001.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

data/repliqa_2-00000-of-00001.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

...

Generating repliqa_2 split:   0%|          | 0/17955 [00:00<?, ? examples/s]

Generating repliqa_3 split:   0%|          | 0/17955 [00:00<?, ? examples/s]

Generating repliqa_4 split:   0%|          | 0/17950 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17950 [00:00<?, ? examples/s]

One-word Q&A pairs found: 223


In [4]:
import random

# Convert to list and shuffle for random splitting
one_word_list = list(one_word_ds)
random.seed(42)  # For reproducibility
random.shuffle(one_word_list)

# Calculate split indices
num_train = 150
num_val = 30
num_test = 40

train_data = one_word_list[:num_train]
val_data = one_word_list[num_train:num_train + num_val]
test_data = one_word_list[num_train + num_val:num_train + num_val + num_test]

# Sanity check (optional)
assert len(set((x['question'], x['answer']) for x in train_data)
           & set((x['question'], x['answer']) for x in val_data)) == 0
assert len(set((x['question'], x['answer']) for x in train_data)
           & set((x['question'], x['answer']) for x in test_data)) == 0
assert len(set((x['question'], x['answer']) for x in val_data)
           & set((x['question'], x['answer']) for x in test_data)) == 0

print(f"Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")


Train: 150 | Val: 30 | Test: 40


In [5]:
import json

with open("repliqa_train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)
with open("repliqa_val.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)
with open("repliqa_test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)


In [7]:
import json
import csv

# Load the JSON data
with open('repliqa_train.json', 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)

# Write to CSV
with open('repliqa_train.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['question', 'answer'])
    writer.writeheader()
    # Select only the 'question' and 'answer' fields
    writer.writerows([{'question': row['question'], 'answer': row['answer']} for row in data])

In [8]:
import json
import csv

# Load the JSON data
with open('repliqa_test.json', 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)

# Write to CSV
with open('repliqa_test.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['question', 'answer'])
    writer.writeheader()
    # Select only the 'question' and 'answer' fields
    writer.writerows([{'question': row['question'], 'answer': row['answer']} for row in data])

In [9]:
import json
import csv

# Load the JSON data
with open('repliqa_val.json', 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)

# Write to CSV
with open('repliqa_val.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['question', 'answer'])
     # Select only the 'question' and 'answer' fields
    writer.writerows([{'question': row['question'], 'answer': row['answer']} for row in data])

TOFU dataset


In [10]:
from datasets import load_dataset

# Download the world facts configuration
dataset = load_dataset("locuslab/TOFU", "world_facts")


README.md: 0.00B [00:00, ?B/s]

world_facts.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/117 [00:00<?, ? examples/s]

In [11]:
def is_one_word(example):
    return (
        isinstance(example["answer"], str) and
        example["answer"].strip() != "" and
        " " not in example["answer"].strip()
    )

one_word_facts = list(filter(is_one_word, dataset["train"]))
print(f"Number of single-word QA pairs: {len(one_word_facts)}")


Number of single-word QA pairs: 67


In [12]:
import random
random.seed(42)
random.shuffle(one_word_facts)

n = len(one_word_facts)
n_train = int(n * 0.6)
n_val = int(n * 0.2)
n_test = n - n_train - n_val

train = one_word_facts[:n_train]
val = one_word_facts[n_train:n_train + n_val]
test = one_word_facts[n_train + n_val:]


In [13]:
import json

with open("tofu_world_facts_train.json", "w", encoding="utf-8") as f:
    json.dump(train, f, ensure_ascii=False, indent=2)
with open("tofu_world_facts_val.json", "w", encoding="utf-8") as f:
    json.dump(val, f, ensure_ascii=False, indent=2)
with open("tofu_world_facts_test.json", "w", encoding="utf-8") as f:
    json.dump(test, f, ensure_ascii=False, indent=2)


In [15]:
import csv

for split_name in ["train", "val", "test"]:
    with open(f"tofu_world_facts_{split_name}.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    with open(f"tofu_world_facts_{split_name}.csv", "w", encoding="utf-8", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["question", "answer"])
        writer.writeheader()
        writer.writerows([{'question': row['question'], 'answer': row['answer']} for row in data])
